In [92]:
import numpy as np

In [93]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 10 
NUM_DIMENSIONS = 2 

In [94]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [95]:
CONSTRAINTS

array([[110, 244],
       [273,  94],
       [108, 201]])

In [178]:
import numpy as np
from icecream import ic
from tqdm import trange 

def cost(solution):
    # checks each column (each item) across all knapsacks
    # returns a boolean vector of length NUM_ITEMS, where each entry is True if that item appears in at least one knapsack
    return VALUES[np.any(solution, axis=0)].sum()

def is_feasible(solution):
    # each item can be at most in one knapsack (= at most 1 True value for each column)
    if np.any(solution.sum(axis=0) > 1):
        return False
    
    total_weights = solution @ WEIGHTS # matrix multiplication (K x N) x (N x D) = (K x D)
    if np.any(total_weights > CONSTRAINTS):
        return False
    return True

def random_solution():
    sol = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool) # NUM_KNAPSACKS x NUM_ITEMS
    items = np.arange(NUM_ITEMS) # [0,1,...,NUM_ITEMS-1]
    np.random.shuffle(items) # random order of items
    for i in items:
        k = np.random.randint(0, NUM_KNAPSACKS)
        sol[:, i] = False # clear that column before adding the item
        sol[k, i] = True
        if not is_feasible(sol):
            sol[k, i] = False  
    return sol

# steepest ascent hill climbing approach 
# generates a random subset of neighbors (not every neighbor too expensive)
def get_neighbors(solution, sample_size=30):
    neighbors = []
    # choose a random subset of items and knapsacks
    items = np.random.choice(NUM_ITEMS, size=sample_size, replace=True)
    knapsacks = np.random.randint(0, NUM_KNAPSACKS, size=sample_size)
    for i, k in zip(items, knapsacks):
        new_sol = solution.copy()
        new_sol[:, i] = False
        new_sol[k, i] = True
        if is_feasible(new_sol):
            neighbors.append(new_sol)
    return neighbors

def hill_climbing(max_iter=100, no_improve_limit=100, neighbor_sample=30):
    current = random_solution() 
    best = current.copy()
    best_val = cost(best)
    no_improve = 0

    for it in trange(max_iter, desc="Hill Climbing"):
        neighbors = get_neighbors(current, sample_size=neighbor_sample)
        if not neighbors:
            break
        # find best improvement among subset of neighbors
        values = [cost(n) for n in neighbors]
        best_idx = np.argmax(values)
        best_neighbor = neighbors[best_idx]
        best_neighbor_val = values[best_idx]
        # if better → move to that solution
        if best_neighbor_val > best_val:
            best = best_neighbor.copy()
            best_val = best_neighbor_val
            current = best_neighbor
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= no_improve_limit:
                break
    return best, best_val

## TEST PROBLEMS

In [184]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# ---------- Run ---------- #
best_solution, best_value = hill_climbing(neighbor_sample=100)
ic(best_value)
ic(best_solution.astype(int))

Hill Climbing: 100%|██████████| 100/100 [00:00<00:00, 629.26it/s]
ic| best_value: 1014
ic| best_solution.astype(int): array([[1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1],
                                      [0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                                      [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]])


array([[1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]])

In [202]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# ---------- Run ---------- #
best_solution, best_value = hill_climbing(neighbor_sample=50)
ic(best_value)
# ic(best_solution.astype(int))

Hill Climbing:  66%|██████▌   | 66/100 [00:00<00:00, 570.72it/s]
ic| best_value: 33037


33037

In [ ]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# ---------- Run ---------- #
best_solution, best_value = hill_climbing(max_iter=50, neighbor_sample=5)
ic(best_value)
# ic(best_solution.astype(int))

""" 
this really runs for long time, knapsack_2 and knapsack_3 are better optimizations 
"""

KeyboardInterrupt: 